# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [2]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [8]:
df_orders=pd.read_csv("Orders.csv")
df_orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [9]:
df_orders = df_orders.drop('Unnamed: 0', axis=1)
df_orders.head()

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [25]:
# Grouping by Customer ID and aggregating the amounts (sum)
df_cust_amt=df_orders.groupby(['CustomerID']).agg({'amount_spent':sum}).reset_index()
df_cust_amt.head()

,CustomerID,amount_spent
1690,14646,280206.02
4202,18102,259657.30
3729,17450,194550.79
3009,16446,168472.50
1880,14911,143825.06
...,...,...
4099,17956,12.75
3015,16454,6.90
1794,14792,6.20
3218,16738,3.75


In [42]:
# Labeling the total amounts falling >95 percentile and between 75-95 percentile
def fill_cust_tier(x):
    if x>np.percentile(df_cust_amt['amount_spent'],95):
        return 'VIP'
    elif x>np.percentile(df_cust_amt['amount_spent'],75) and x<=np.percentile(df_cust_amt['amount_spent'],95):
        return 'Preferred'
    else:
        return 'Unknown'

df_cust_amt['Customer_Tier']=df_cust_amt['amount_spent'].apply(fill_cust_tier)
df_cust_amt.head()

,CustomerID,amount_spent,Customer_Tier
0,12346,77183.60,VIP
1,12347,4310.00,Preferred
2,12348,1797.24,Preferred
3,12349,1757.55,Preferred
4,12350,334.40,Unknown


In [44]:
# Joining the 2 DFs
df_orders_cust_tiers = pd.merge(left = df_orders,
                                 right = df_cust_amt,
                                 how = 'inner', 
                                 left_on = "CustomerID", 
                                 right_on= "CustomerID")

df_orders_cust_tiers = df_orders_cust_tiers.rename(columns={'amount_spent_x': 'amount_spent', 'amount_spent_y': 'total_amount_spent'})
df_orders_cust_tiers.head()

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,total_amount_spent,Customer_Tier
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,5391.21,Preferred
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,5391.21,Preferred
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred


In [43]:
# Check the labeling in Customer Tier column
df_cust_amt['Customer_Tier'].value_counts()

Unknown      3254
Preferred     868
VIP           217
Name: Customer_Tier, dtype: int64

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [49]:
df_orders_cust_tiers[df_orders_cust_tiers['Customer_Tier']=='VIP']['Country'].mode().iloc[0]

'United Kingdom'

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [51]:
df_orders_cust_tiers[(df_orders_cust_tiers['Customer_Tier']=='VIP') | (df_orders_cust_tiers['Customer_Tier']=='Preferred')]['Country'].mode().iloc[0]

'United Kingdom'